In [ ]:
import yfinance as yf
import numpy as np
import matplotlib.pyplot as plt
from polygon import RESTClient
import dotenv
import os
import pandas as pd
import warnings

warnings.filterwarnings('ignore')

dotenv.load_dotenv()
POLYGON_API = os.getenv("POLYGON_API")

client = RESTClient(api_key=POLYGON_API)

In [40]:
'''Taking data from spy stocks to build the strat around'''
#We parse the wikipedia page for spy data

sp500 = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]
sp500['Symbol'] = sp500['Symbol'].str.replace('.', '-')
tickers = sp500['Symbol'].unique().tolist()

data = yf.download(tickers, period='1y')
data = data.stack()
data.index.names = ['date', 'ticker']
data.columns = data.columns.str.lower()

[*********************100%***********************]  503 of 503 completed


In [82]:
'''Well now filter by dollar volume to only opearte with the stocks which fit our liquidity constraints'''
#We will compute the mean dollar volume from the past year to filter liquid stocks

data['dollarvol'] = (data['close'] * data['volume']) / 1e6
data = data.groupby('ticker').mean() #finding mean dollar vol over past year

data = data.sort_values(by='dollarvol', ascending = False)
data = data.head(100) #top 100 liquid stocks

In [ ]:
'''Now we download the data for the 100 stocks we selected'''

tickers = " ".join(data.index.tolist())

options_data = yf.download(tickers, period="1y")
options_data = options_data.stack()
options_data.index.names = ['date', 'ticker']
options_data.columns = options_data.columns.str.lower()

[*********************100%***********************]  100 of 100 completed


In [100]:
options_data

Price                   close        high         low        open      volume
date       ticker                                                            
2024-03-08 AAPL    169.927628  172.883671  168.146047  168.205762  76114600.0
           ABBV    172.563919  175.738278  172.399881  173.422637   6195600.0
           ABNB    164.910004  168.190002  163.479996  166.000000   4103600.0
           ABT     118.544167  119.210586  118.201158  118.553969   3357100.0
           ACN     373.546021  382.771814  373.111398  381.961833   1641100.0
...                       ...         ...         ...         ...         ...
2025-03-07 VZ       46.060001   46.200001   44.340000   44.299999  44972510.0
           WDAY    252.380005  255.617493  245.449997  251.350006   1963609.0
           WFC      71.050003   72.160004   68.099998   72.250000  18959119.0
           WMT      91.720001   93.949997   90.760002   93.940002  28160238.0
           XOM     109.019997  110.400002  108.300003  108.410004  15490046.0

[24985 rows x 5 columns]